In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_excel('tarrodan_dam.csv')

print("Shape of data:", df.shape)
print("\nMissing values in each column:")
print(df.isnull().sum())

def clean_assessment(value):
    if pd.isna(value) or value == 'undefined':
        return 'Not Available'
    return value

df['Assessment'] = df['Assessment'].apply(clean_assessment)

In [ ]:
def clean_date(date_str):
    if pd.isna(date_str):
        return np.nan
    try:
        return pd.to_datetime(date_str)
    except:
        return np.nan

df['Last Inspection Date'] = df['Last Inspection Date'].apply(clean_date)
df['Assessment Date'] = df['Assessment Date'].apply(clean_date)

print("\nUnique hazard values before cleaning:")
print(df['Hazard'].value_counts())

df['Hazard'] = df['Hazard'].replace('Undetermined', 'Low')

numeric_cols = ['Height (m)', 'Length (km)', 'Volume (m3)', 
                'Surface (km2)', 'Drainage (km2)', 'Probability of Failure']

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df.to_excel('tarrodan_dam_cleaned.xlsx', index=False)

print("\nCleaning Summary:")
print("1. Standardized Assessment ratings")
print("2. Cleaned date formats")
print("3. Handled Undetermined hazard cases")
print("4. Converted numeric columns to proper type")

In [ ]:
df = pd.read_excel('tarrodan_dam.csv')

df['Assessment'] = df['Assessment'].fillna('Not Available')
df['Assessment'] = df['Assessment'].replace('undefined', 'Not Available')

df['Last Inspection Date'] = pd.to_datetime(df['Last Inspection Date'], errors='coerce')
df['Assessment Date'] = pd.to_datetime(df['Assessment Date'], errors='coerce')

df['Hazard'] = df['Hazard'].replace('Undetermined', 'Low')

numeric_cols = ['Height (m)', 'Length (km)', 'Volume (m3)', 'Surface (km2)', 'Drainage (km2)']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    median_by_region = df.groupby('Region')[col].transform('median')
    df[col] = df[col].fillna(median_by_region)

loss_cols = ['Loss given failure - prop (Qm)', 'Loss given failure - liab (Qm)', 'Loss given failure - BI (Qm)']
for col in loss_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    median_by_hazard = df.groupby('Hazard')[col].transform('median')
    df[col] = df[col].fillna(median_by_hazard)

df['Years Since Inspection'] = (pd.Timestamp.now() - df['Last Inspection Date']).dt.total_seconds() / (365.25 * 24 * 60 * 60)
df['Years Since Inspection'] = df['Years Since Inspection'].fillna(df['Years Since Inspection'].median())

df['Total Loss Given Failure'] = df['Loss given failure - prop (Qm)'] + df['Loss given failure - liab (Qm)'] + df['Loss given failure - BI (Qm)']

df['Risk Score'] = df['Probability of Failure'] * df['Total Loss Given Failure']

df['Hazard_Numeric'] = df['Hazard'].map({'Low': 1, 'Significant': 2, 'High': 3})

df.to_excel('tarrodan_dam_cleaned_v2.xlsx', index=False)

print("Missing values after cleaning:")
print(df.isnull().sum())

print("\nRisk Score Summary by Region:")
print(df.groupby('Region')['Risk Score'].describe())

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.impute import SimpleImputer

df = pd.read_excel('tarrodan_dam_cleaned_v2.xlsx')

# Create more sophisticated features
df['Age'] = 2025 - pd.to_numeric(df['Year Completed'], errors='coerce')
df['Maintenance_Score'] = pd.Categorical(df['Assessment']).codes  # Convert assessment to numeric
df['Population_Risk'] = df['Hazard_Numeric'] * (1 / (df['Distance to Nearest City (km)'] + 1))
df['Volume_Risk'] = df['Volume (m3)'] * df['Height (m)'] / 1000000  # Normalized volume risk

# Separate losses by type for more detailed modeling
df['Property_Loss_Weight'] = df['Loss given failure - prop (Qm)'] / df['Total Loss Given Failure']
df['Liability_Loss_Weight'] = df['Loss given failure - liab (Qm)'] / df['Total Loss Given Failure']
df['Business_Loss_Weight'] = df['Loss given failure - BI (Qm)'] / df['Total Loss Given Failure']

features = [
    'Height (m)',
    'Volume (m3)',
    'Hazard_Numeric',
    'Surface (km2)',
    'Age',
    'Maintenance_Score',
    'Population_Risk',
    'Volume_Risk',
    'Property_Loss_Weight',
    'Liability_Loss_Weight',
    'Business_Loss_Weight'
]

X = df[features]
y = df['Total Loss Given Failure'] * df['Probability of Failure']

# Handle missing values
imputer = SimpleImputer(strategy='median')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = GradientBoostingRegressor(
    n_estimators=200,
    max_depth=5,
    min_samples_split=20,
    learning_rate=0.05,
    random_state=42
)

model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

print("Model Performance:")
print(f"R2 Score: {r2_score(y_test, y_pred):.4f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.4f}")

feature_importance = pd.DataFrame({
    'feature': features,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nFeature Importance:")
print(feature_importance)